In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('/Users/localadmin/Desktop/cleaned_multiclass_dataset.csv')
df.head()

,cleaned_text,label
0,"['word', 'food', 'crapilicious']",0
1,['white'],0
2,"['classy', 'whore', 'red', 'velvet', 'cupcake']",0
3,"['meh', 'p', 'thanks', 'head', 'concerned', 'a...",0
4,"['isi', 'account', 'pretending', 'kurdish', 'a...",0


In [5]:
df['label'].value_counts()

label
1    7998
2    7992
3    7973
4    7961
0    7945
5    7823
Name: count, dtype: int64

In [13]:
x= df['cleaned_text']
y= df['label']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3,stratify=y, random_state=42)
print(x_train.shape,x_test.shape, y_train.shape,y_test.shape)

(31520,) (13509,) (31520,) (13509,)


In [8]:
from imblearn.over_sampling import RandomOverSampler

In [9]:
df["cleaned_text"].duplicated().sum()

2663

In [10]:
df.drop_duplicates("cleaned_text", inplace=True)


In [11]:
df.label.value_counts()

label
1    7963
2    7905
4    7757
0    7742
3    7721
5    5941
Name: count, dtype: int64

In [15]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_length = 5000
max_features = 20000
embedding_dim = 300

x_all = []
x_all.extend(x_test)
x_all.extend(x_train)

tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(x_train)
x_test_seq = tk.texts_to_sequences(x_test)

np_x_train = pad_sequences(x_train_seq, maxlen=max_length,  padding='post')
np_x_test = pad_sequences(x_test_seq, maxlen=max_length,  padding='post')
np_y_train = to_categorical(y_train)

class_num = np_y_train.shape[1]

print ('np_x_train shape: {}'.format(np_x_train.shape))
print ('np_x_test shape: {}'.format(np_x_test.shape))
print ('np_y_train shape: {}'.format(np_y_train.shape))

2024-01-14 20:53:22.410502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


np_x_train shape: (31520, 5000)
np_x_test shape: (13509, 5000)
np_y_train shape: (31520, 6)


In [16]:
from imblearn.over_sampling import RandomOverSampler

# Reshape np_x_train to 2D array
np_x_train_2d = np_x_train.reshape((np_x_train.shape[0], -1))

# Instantiate RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Fit and apply the oversampler to the training data
np_x_train_resampled, np_y_train_resampled = ros.fit_resample(np_x_train_2d, np_y_train.argmax(axis=1))

# Reshape back to 3D array
np_x_train_resampled = np_x_train_resampled.reshape((-1, max_length, np_x_train.shape[2]))

# Convert labels back to categorical
np_y_train_resampled = to_categorical(np_y_train_resampled, num_classes=class_num)

print ('np_x_train_resampled shape: {}'.format(np_x_train_resampled.shape))
print ('np_y_train_resampled shape: {}'.format(np_y_train_resampled.shape))


IndexError: tuple index out of range

In [13]:
class_num = y_train_resampled.shape[1]

In [17]:
from imblearn.over_sampling import SMOTE
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Assuming you have x_train, y_train as your original data
max_length = 5000
max_features = 20000
embedding_dim = 300

# Combine text data for tokenization
x_all = x_train.copy()

# Tokenize and pad sequences
tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(x_train)
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post')

# Convert labels to categorical
y_train_categorical = to_categorical(y_train)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_pad, y_train_categorical.argmax(axis=1))

# Reshape the resampled data back to 3D array
x_train_resampled = x_train_resampled.reshape((-1, max_length, 1))  # Assuming 1 as the embedding_dim

# Convert labels back to categorical
y_train_resampled = to_categorical(y_train_resampled, num_classes=6)

# Now you can use x_train_resampled and y_train_resampled in your CNN model
print('x_train_resampled shape:', x_train_resampled.shape)
print('y_train_resampled shape:', y_train_resampled.shape)


x_train_resampled shape: (33444, 5000, 1)
y_train_resampled shape: (33444, 6)


In [ ]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping

def one_input_classifier(max_length, max_features, embedding_dim, class_num):
    inputs = Input(shape=(max_length,), name='input_1')
    embeddings = Embedding(max_features, embedding_dim, input_length=max_length, name='embedding_1')(inputs)

    conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(embeddings)
    maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
    dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)

    conv_2 = Conv1D(32, 7, activation='relu', name='conv1d_2')(dropout_1)
    maxpool_2 = MaxPooling1D(8, name='maxpool1d_2')(conv_2)
    dropout_2 = Dropout(0.2, name='dropout_2')(maxpool_2)

    bilstm = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm_1'),
        name='bidirectional_1')(dropout_2)
    preds = Dense(class_num, activation='softmax', name='preds')(bilstm)

    model = Model(inputs=inputs, outputs=preds)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

models = []
classifier_num = 10

for i in range(classifier_num):
    model = one_input_classifier(max_length, max_features, embedding_dim, class_num)

    if i == 0:
        print(model.summary())

    model.fit(x_train_resampled, y_train_resampled, validation_split=0.3, shuffle=True, epochs=2, batch_size=32, verbose=1)
    models.append(model)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5000)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 5000, 300)         6000000   
                                                                 
 conv1d_1 (Conv1D)           (None, 4992, 32)          86432     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 312, 32)           0         
                                                                 
 dropout_1 (Dropout)         (None, 312, 32)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 306, 32)           7200      
                                                                 
 maxpool1d_2 (MaxPooling1D)  (None, 38, 32)            0     

In [2]:
from sklearn.metrics import classification_report

# Assuming model is the last model in your 'models' list
last_model = models[-1]

# Assuming x_test and y_test are your test data
y_pred = last_model.predict(x_test)

# Convert probabilities to class labels
y_pred_classes = (y_pred > 0.5).astype(int)

# Assuming binary classification
print(classification_report(y_test, y_pred_classes))


NameError: name 'model' is not defined